In [1]:
import os

In [2]:
%pwd

'/Users/bhikipallai/Desktop/Projects/95Mobiles/research/bike'

In [3]:
os.chdir("../../")

In [4]:
%pwd

'/Users/bhikipallai/Desktop/Projects/95Mobiles'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class BikeModelTrainingConfig:
    root_dir: Path
    X_train: Path
    y_train: Path
    save_model: Path
    save_model_for_train: Path
    n_estimator: int
    criterion: str

In [6]:
from src.bike.utils.common import read_yaml,create_directory
from src.bike.constants import *

[2024-09-29 21:29:33,371: INFO]: Logging file start


In [7]:
class ConfigurationManger:
    def __init__(self,config_filepath = CONFIG_FILE_PATH,params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directory([self.config.data_root])

    def get_model_training(self) -> BikeModelTrainingConfig:
        config = self.config.model_training
        params = self.params.model

        create_directory([config.root_dir])

        model_train = BikeModelTrainingConfig(
            root_dir=config.root_dir,
            X_train = config.X_train,
            y_train= config.y_train,
            save_model = config.save_model,
            save_model_for_train = config.save_model_for_train,
            n_estimator = params.n_estimators,
            criterion = params.criterion
        )

        return model_train

In [8]:
from sklearn.ensemble import RandomForestRegressor
import pandas as pd
import joblib

In [11]:
class ModelTrain:
    def __init__(self,config: BikeModelTrainingConfig):
        self.config = config
    
    def mode_train(self):
        rfr = RandomForestRegressor(n_estimators=self.config.n_estimator,criterion=self.config.criterion)
        X_train = pd.read_csv(self.config.X_train)
        
        print("this is for X_train:\n",X_train.head(2))
        y_train = pd.read_csv(self.config.y_train)
        print("this is for X_train:\n",y_train.head(2))
        model = rfr.fit(X_train,y_train)

        return model
    
    def save_model(self):
        model = self.model_train()
        model_path = self.config.save_model
        joblib.dump(model,model_path)


In [12]:
try:
    config = ConfigurationManger()
    model_config = config.get_model_training()
    mt = ModelTrain(config=model_config)
    mt.mode_train()
    mt.save_model()
except Exception as e:
    raise e

[2024-09-29 22:36:44,705: INFO]: yaml: config/bike_config.yaml loaded successfully
[2024-09-29 22:36:44,709: INFO]: yaml: params/bike_params.yaml loaded successfully
[2024-09-29 22:36:44,712: INFO]: created directory at: data/bike
[2024-09-29 22:36:44,715: INFO]: created directory at: data/bike/model_training
this is for X_train:
    Unnamed: 0  bike_name  city  kms_driven  owner   age    power
0           0        151     3   -0.645916      0   6.0 -0.84441
1           1         56    10    0.434766      0  15.0 -0.47246
this is for X_train:
      price
0  32000.0
1  14227.0


/Applications/anaconda3/envs/95m/lib/python3.12/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


AttributeError: 'ModelTrain' object has no attribute 'model_train'